In [1]:
import sys
sys.path.append('../..')

In [2]:
from skactiveml.pool import RandomSampler

In [27]:
RandomSampler(batch_size=2, random_state=1).query([[1],[1],[1],[1],[1]], return_utilities=False)

array([1, 0])